In [2]:
import sys, os
sys.path.append('../')
import numpy as np
from numba import jit, njit, float32, boolean, int32, float64
from utils.config import cfg
from utils.tools.evaluation import dBZ_to_pixel
from utils.tools.numba_accelerated import *
import numpy as np
from utils.config import cfg
import matplotlib as mpl
import matplotlib.pyplot as plt
from utils.tools.evaluation import get_GDL, get_hit_miss_counts, get_balancing_weights
from numpy.testing import assert_allclose, assert_almost_equal
import time

In [3]:
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')

In [4]:
prediction = np.random.uniform(size=(10, 16, 1, 480, 480))
truth = np.random.uniform(size=(10, 16, 1, 480, 480))
mask = np.random.randint(low=0, high=2, size=(10, 16, 1, 480, 480)).astype(np.bool)

begin = time.time()
gdl = get_GDL(prediction=prediction, truth=truth, mask=mask)
end = time.time()
print("numpy gdl:", end - begin)

begin = time.time()
gdl_numba = get_GDL_numba(prediction=prediction, truth=truth, mask=mask)
end = time.time()
print("numba gdl:", end - begin)

# gdl_mx = mx_get_GDL(prediction=prediction, truth=truth, mask=mask)
# print gdl_mx
assert_allclose(gdl, gdl_numba, rtol=1E-4, atol=1E-3)

begin = time.time()
for i in range(5):
    hits, misses, false_alarms, true_negatives = get_hit_miss_counts(prediction, truth, mask)
end = time.time()
print("numpy hits misses:", end - begin)

begin = time.time()
for i in range(5):
    hits_numba, misses_numba, false_alarms_numba, true_negatives_numba = get_hit_miss_counts_numba(prediction, truth, mask)
end = time.time()
print("numba hits misses:", end - begin)

print(np.abs(hits - hits_numba).max())
print(np.abs(misses - misses_numba).max(), np.abs(misses - misses_numba).argmax())
print(np.abs(false_alarms - false_alarms_numba).max(),
        np.abs(false_alarms - false_alarms_numba).argmax())
print(np.abs(true_negatives - true_negatives_numba).max(),
        np.abs(true_negatives - true_negatives_numba).argmax())

begin = time.time()
for i in range(5):
    weights_npy = get_balancing_weights(data=truth, mask=mask,
                                        base_balancing_weights=None, thresholds=None)
end = time.time()
print("numpy balancing weights:", end - begin)

begin = time.time()
for i in range(5):
    weights_numba = get_balancing_weights_numba(data=truth, mask=mask,
                                                base_balancing_weights=None, thresholds=None)
end = time.time()
print("numba balancing weights:", end - begin)
print("Inconsistent Number:", (np.abs(weights_npy - weights_numba) > 1E-5).sum())

numpy gdl: 1.0690011978149414
numba gdl: 1.220999002456665
numpy hits misses: 26.3617160320282
numba hits misses: 4.7290942668914795
1
1 116
0 0
0 0
numpy balancing weights: 15.002631902694702
numba balancing weights: 4.119091510772705
Inconsistent Number: 0
